Korelacja Spearmana dla indexów:
S&P500 (^GSPC),
DAX (^GDAXI),
Nikkei225 (^N225),
FTSE100 (^FTSE),
Shanghai Composite (000001.SS)

In [13]:
pip install yfinance==0.2.59 

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.54
    Uninstalling yfinance-0.2.54:
      Successfully uninstalled yfinance-0.2.54
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\xxx\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
data = yf.download(['^GSPC','^GDAXI', '^FTSE', '^N225', '000001.SS'], start= '2000-01-01')['Close']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  5 of 5 completed


In [4]:
data

Ticker,000001.SS,^FTSE,^GDAXI,^GSPC,^N225
Date,,,,,
2000-01-03,NaN,NaN,6750.759766,1455.219971,NaN
2000-01-04,1406.370972,6665.899902,6586.950195,1399.420044,19002.859375
2000-01-05,1409.682007,6535.899902,6502.069824,1402.109985,18542.550781
2000-01-06,1463.942017,6447.200195,6474.919922,1403.449951,18168.269531
2000-01-07,1516.604004,6504.799805,6780.959961,1441.469971,18193.410156
...,...,...,...,...,...
2025-05-06,3316.114014,8597.400391,23249.650391,5606.910156,NaN
2025-05-07,3342.665039,8559.299805,23115.960938,5631.279785,36779.660156
2025-05-08,3351.996094,8531.599609,23352.689453,5663.939941,36928.628906


In [5]:
data_log = np.log(data/data.shift(1)).dropna()

In [17]:
cor = data_log.corr(method='spearman')
cor


Ticker,000001.SS,^FTSE,^GDAXI,^GSPC,^N225
Ticker,,,,,
000001.SS,1.000000,0.117208,0.097087,0.065496,0.208469
^FTSE,0.117208,1.000000,0.779923,0.473689,0.261532
^GDAXI,0.097087,0.779923,1.000000,0.526215,0.256862
^GSPC,0.065496,0.473689,0.526215,1.000000,0.124583
^N225,0.208469,0.261532,0.256862,0.124583,1.000000


In [18]:
df = pd.DataFrame(cor)
df.to_excel("korelacjatabela.xlsx", index=False)

Korelacja krocząca

In [8]:
import plotly.graph_objects as go

In [11]:
tickers = {
    '^GSPC': 'S&P 500',
    '000001.SS': 'Shanghai Composite',
    '^FTSE': 'FTSE 100',
    '^GDAXI': 'DAX',
    '^N225': 'Nikkei 225'
}

In [12]:
rolling_corrs = pd.DataFrame(index=data_log.index)
reference = '^GSPC'
window = 252

for symbol in tickers:
    if symbol != reference:
        rolling_corrs[tickers[symbol]] = data_log[reference].rolling(window).corr(data_log[symbol])


In [13]:
fig = go.Figure()

for col in rolling_corrs.columns:
    fig.add_scatter(x=rolling_corrs.index, y=rolling_corrs[col], mode='lines', name=f'{col} vs S&P 500')

fig.update_layout(
    title='Korelacje kroczące 252-dniowe względem S&P 500 (od 2000)',
    xaxis_title='Data',
    yaxis_title='Wartość korelacji',
    template='plotly_dark',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
)

fig.show()

Współczynnik zmienności

In [7]:
data.std()/data.mean()

Ticker
000001.SS    0.325439
^FTSE        0.183363
^GDAXI       0.477825
^GSPC        0.604185
^N225        0.454308
dtype: float64